This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/wallaroo-model-cookbooks/demand_curve).

## Demand Curve Pipeline Tutorial

This worksheet demonstrates a Wallaroo pipeline with data preprocessing, a model, and data postprocessing.

The model is a "demand curve" that predicts the expected number of units of a product that will be sold to a customer as a function of unit price and facts about the customer. Such models can be used for price optimization or sales volume forecasting.  This is purely a "toy" demonstration, but is useful for detailing the process of working with models and pipelines.

Data preprocessing is required to create the features used by the model. Simple postprocessing prevents nonsensical estimates (e.g. negative units sold).

## Prerequisites

* An installed Wallaroo instance.
* The following Python libraries installed:
  * `os`
  * [`wallaroo`](https://pypi.org/project/wallaroo/): The Wallaroo SDK. Included with the Wallaroo JupyterHub service by default.

In [1]:
import json
import wallaroo
from wallaroo.object import EntityNotFoundError
import pandas
import numpy
import conversion
from wallaroo.object import EntityNotFoundError

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [3]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

wallarooPrefix = "doc-test"
wallarooSuffix = "wallaroocommunity.ninja"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
                    auth_type="sso")

Now that the Wallaroo client has been initialized, we can create the workspace and call it `demandcurveworkspace`, then set it as our current workspace.  We'll also create our pipeline so it's ready when we add our models to it.

We'll set some variables and methods to create our workspace, pipelines and models.  Note that as of the July 2022 release of Wallaroo, workspace names must be unique.  Pipelines with the same name will be created as a new version when built.

In [4]:
workspace_name = 'demandcurveworkspace'
pipeline_name = 'demandcurvepipeline'
model_name = 'demandcurvemodel'
model_file_name = './demand_curve_v1.onnx'

In [5]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

In [6]:
workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

demandcurve_pipeline = get_pipeline(pipeline_name)
demandcurve_pipeline

name,demandcurvepipeline
created,2023-07-06 16:13:32.024927+00:00
last_updated,2023-07-06 16:38:54.679586+00:00
deployed,False
tags,
versions,"137f0f42-9c4c-4427-9e6e-ff78a53e8301, fa1fe10d-94a2-4227-8665-6bf814089b28, 19c008da-6c0e-4307-b652-d43cc2eba0a6"
steps,preprocess


With our workspace established, we'll upload three models:

* `demand_curve_v1.onnx`: Our demand_curve model.  We'll store the upload configuration into `demand_curve_model`.
* `preprocess`:  Takes the data and prepares it for the demand curve model.  We'll store the upload configuration into `module_pre`.
* `postprocess`:  Takes the results from our demand curve model and prepares it for our display.  We'll store the upload configuration into `module_post`.

Note that the order we upload our models isn't important - we'll be establishing the actual process of moving data from one model to the next when we set up our pipeline.

In [7]:
# upload to wallaroo
demand_curve_model = wl.upload_model(model_name, model_file_name).configure()

In [16]:
# load the preprocess module
module_pre = wl.upload_model("preprocess", "./preprocess.py").configure('python')

In [17]:
# load the postprocess module
module_post = wl.upload_model("postprocess", "./postprocess.py").configure('python')

With our models uploaded, we're going to create our own pipeline and give it three steps:

* First, start with the preprocess module we called `module_pre` to prepare the data.
* Second, we apply the data to our `demand_curve_model`.
* And finally, we prepare our data for output with the `module_post`.

In [18]:
# now make a pipeline
demandcurve_pipeline.clear()
demandcurve_pipeline.add_model_step(module_pre)
demandcurve_pipeline.add_model_step(demand_curve_model)
demandcurve_pipeline.add_model_step(module_post)

name,demandcurvepipeline
created,2023-07-06 16:13:32.024927+00:00
last_updated,2023-07-06 16:42:43.522606+00:00
deployed,False
tags,
versions,"c634d4f7-f350-4321-9360-6ca011e40bd8, 137f0f42-9c4c-4427-9e6e-ff78a53e8301, fa1fe10d-94a2-4227-8665-6bf814089b28, 19c008da-6c0e-4307-b652-d43cc2eba0a6"
steps,preprocess


And with that - let's deploy our model pipeline.  This usually takes about 45 seconds for the deployment to finish.

In [19]:
demandcurve_pipeline.deploy()

name,demandcurvepipeline
created,2023-07-06 16:13:32.024927+00:00
last_updated,2023-07-06 16:45:12.652538+00:00
deployed,True
tags,
versions,"7c492cdb-d2f1-4cec-8335-95ae82e8c511, c634d4f7-f350-4321-9360-6ca011e40bd8, 137f0f42-9c4c-4427-9e6e-ff78a53e8301, fa1fe10d-94a2-4227-8665-6bf814089b28, 19c008da-6c0e-4307-b652-d43cc2eba0a6"
steps,preprocess


We can check the status of our pipeline to make sure everything was set up correctly:

In [20]:
demandcurve_pipeline.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.244.3.114',
   'name': 'engine-6c4bb7f655-tzpr8',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'demandcurvepipeline',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'demandcurvemodel',
      'version': '59e0bf0d-3ca6-4af8-b5d4-a6e4136d6f88',
      'sha': '2820b42c9e778ae259918315f25afc8685ecab9967bad0a3d241e6191b414a0d',
      'status': 'Running'},
     {'name': 'preprocess',
      'version': '4e610a36-79a2-4a88-85fe-85c976e96dd2',
      'sha': 'd952371d541b216adbc9e2fb4e7e25e673bac5d0b2a38505c7d4040df82957fc',
      'status': 'Running'},
     {'name': 'postprocess',
      'version': 'da0f20c9-4bf7-481c-ab43-d75a200df8eb',
      'sha': '0649e62625b68a96c722d0e05d8e28fb58ddc96c99543f78039d327896325a54',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.244.4.159',
   'name': 'engine-lb-584f54c899-pjgww',
   'status': 'Running',
   

Everything is ready.  Let's feed our pipeline some data.  We have some information prepared with the `daily_purchasses.csv` spreadsheet.  We'll start with just one row to make sure that everything is working correctly.

In [13]:
# read in some purchase data
purchases = pandas.read_csv('daily_purchases.csv')

# start with a one-row data frame for testing
subsamp_raw = purchases.iloc[0:1,: ]
subsamp_raw

# create the input dictionary from the original one-line dataframe
input_dict = conversion.pandas_to_dict(subsamp_raw)

In [21]:
result = demandcurve_pipeline.infer(input_dict)

/opt/homebrew/anaconda3/envs/wallaroosdk2023.2.1/lib/python3.9/site-packages/wallaroo/deployment.py:532: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tensor[col] = flatten_np_array_columns(tensor, col)


InferenceError: Inference failed: 
	InternalError: Error running inference {"error": "Python: Exception: <class 'AttributeError'>, 'str' object has no attribute 'copy'"}

In [ ]:
display(result)

We can see from the `prediction` field that the demand curve has a predicted slope of 6.68 from our sample data.  We can isolate that by specifying just the data output below.

In [14]:
display(result[0]['prediction'])

[6.68025518653071]

# Bulk Inference

The initial test went perfectly.  Now let's throw some more data into our pipeline.  We'll draw 10 random rows from our spreadsheet, perform an inference from that, and then display the results and the logs showing the pipeline's actions.

In [15]:
# Let's do 10 rows at once (drawn randomly)
ix = numpy.random.choice(purchases.shape[0], size=10, replace=False)
output = demandcurve_pipeline.infer(conversion.pandas_to_dict(purchases.iloc[ix,: ]))

In [16]:
display(output[0]['prediction'])

[40.57067889202563,
 6.68025518653071,
 40.57067889202563,
 9.110871146224234,
 33.125323160373426,
 33.125323160373426,
 33.125323160373426,
 40.57067889202563,
 6.771545926800889,
 40.57067889202563]

## Undeploy the Pipeline

Once we've finished with our demand curve demo, we'll undeploy the pipeline and give the resources back to our Kubernetes cluster.

In [22]:
demandcurve_pipeline.undeploy()

name,demandcurvepipeline
created,2023-07-06 16:13:32.024927+00:00
last_updated,2023-07-06 16:45:12.652538+00:00
deployed,False
tags,
versions,"7c492cdb-d2f1-4cec-8335-95ae82e8c511, c634d4f7-f350-4321-9360-6ca011e40bd8, 137f0f42-9c4c-4427-9e6e-ff78a53e8301, fa1fe10d-94a2-4227-8665-6bf814089b28, 19c008da-6c0e-4307-b652-d43cc2eba0a6"
steps,preprocess


Thank you for being a part of this demonstration.  If you have additional questions, please feel free to contact us at Wallaroo.